In [14]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd 
from datetime import date,datetime,timedelta
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import os
import sys

In [15]:
companies_list = pd.read_csv("company_name.csv")
companies_list.head()

,Co Names
0,Apple Inc
1,Microsoft Corp
2,Alphabet Inc
3,Amazon.com Inc
4,Berkshire Hathaway Inc


In [16]:
len(companies_list)

1997

In [17]:
companies_list_second = companies_list[1000:-1]

In [18]:
companies_list = companies_list[:1000]

In [20]:
search_form_keys_template = r'ns=ntra and rst=fndw and hd=("Earnings Call - Final" and "QQ" and "Event Brief")'
search_form_keys_earning_calls_only_template = r'hd=("Earnings Call - Final" and "QQ") not "Event Brief" and ns=ntra and rst=fndw'
quarters = ['Q1','Q2','Q3','Q4']
search_form_ID = r'ftx'
date_select_ID = r'dr'
date_select_text = r'Enter date range...'
date_from_month_ID = r'frm'
date_from_day_ID = r'frd'
date_from_year_ID = r'fry'
date_to_month_ID = r'tom'
date_to_day_ID = r'tod'
date_to_year_ID = r'toy'
company_arrow_XPATH = r'//*[@id="coTab"]/div[1]'
company_promote_XPATH = r'//*[@id="coMnu"]/ul/li[1]/a[3]'
company_text_ID = r'coTxt'
search_button_ID = r'btnSBSearch'
select_all_files_button_XPATH =r'//*[@id="selectAll"]/input'
pdf_button_XPATH = r'//*[@id="listMenuRoot"]/li[6]/a'
headline_button_XPATH = r'//*[@id="listMenu-id-4"]/li[3]/a'

In [21]:
is_earning_calls_only = False

In [22]:
chrome_options = webdriver.ChromeOptions()
prefs = {'download.default_directory' : r"": False,"disable-features":"DownloadUI"}
chrome_options.add_experimental_option('prefs', prefs)
driver = webdriver.Chrome(options=chrome_options)
driver.get("http://proxy.uchicago.edu/login?url=https://global.factiva.com/en/sess/login.asp?XSID=S003Wv90Gb95DEs5DEnODYqMDEsND7yMHn0VrJoWXNG2TBX4cjBQUFBQUFBQUFBQUFBQUFBQUFBQUFBQUFBQUFBQUFBQQAA")

In [23]:
downloads_tracker = {
    'company' : [],
    'from_date' : [],
    'to_date' : [],
    'status' : [],
    'reason' : []
}

In [24]:
for i in range(len(companies_list)):
    storage_path = r"" + str(companies_list.iloc[i,0]).replace("/","")
    if not os.path.exists(storage_path):
        os.mkdir(storage_path)
    for quarter in quarters:
        status = False
        reason = ""
        driver.get("http://proxy.uchicago.edu/login?url=https://global.factiva.com/en/sess/login.asp?XSID=S003Wv90Gb95DEs5DEnODYqMDEsND7yMHn0VrJoWXNG2TBX4cjBQUFBQUFBQUFBQUFBQUFBQUFBQUFBQUFBQUFBQUFBQQAA")
        company = companies_list.iloc[i,0]
        try:
            while driver.find_element(By.ID,date_select_ID) == []:
                pass    
            
            # selecting date
            date_selector = Select(driver.find_element(By.ID,date_select_ID))
            date_selector.select_by_visible_text(date_select_text)
            
            #Selecting From Date
            month_input = driver.find_element(By.ID,date_from_month_ID)
            day_iput = driver.find_element(By.ID,date_from_day_ID)
            year_input = driver.find_element(By.ID,date_from_year_ID)
            month_input.send_keys("01")
            day_iput.send_keys("01")
            year_input.send_keys("2008")
            
            #Selecting To Date
            month_input = driver.find_element(By.ID,date_to_month_ID)
            day_iput = driver.find_element(By.ID,date_to_day_ID)
            year_input = driver.find_element(By.ID,date_to_year_ID)
            month_input.send_keys("01")
            day_iput.send_keys("01")
            year_input.send_keys("2024")
            
            # Clicking the company button
            company_button = driver.find_element(By.XPATH,company_arrow_XPATH)
            company_button.click()

            # # Selecting the company
            time.sleep(2)
            company_form = driver.find_element(By.ID,company_text_ID)
            search_form = driver.find_element(By.ID,search_form_ID)
            company_form.send_keys(company)
            time.sleep(2)
            company_form.send_keys(Keys.DOWN,Keys.ENTER)
            
            
            # Setting the Search form textbox 
            search_form = driver.find_element(By.ID,search_form_ID)
            if is_earning_calls_only:
                search_form_keys_earning_calls_only = search_form_keys_earning_calls_only_template.replace("QQ",quarter)
                search_form.send_keys(search_form_keys_earning_calls_only)
            else:
                search_form_keys = search_form_keys_template.replace("QQ",quarter)
                search_form.send_keys(search_form_keys)

            # Clicking the search button
            search_button = driver.find_element(By.ID,search_button_ID)
            search_button.click()
            #Selecting all files
            time.sleep(2)
            select_all_files_checkbox_search = driver.find_elements(By.XPATH,select_all_files_button_XPATH)
            if select_all_files_checkbox_search == []:
                status = False
                reason = "No Files"
            else:
                select_all_files_checkbox = driver.find_element(By.XPATH,select_all_files_button_XPATH)
                select_all_files_checkbox.click()
                time.sleep(2)
                # Downloading All  Files
                new_file_name = str(companies_list.iloc[i,0]).replace("/","") + "_20080101_20240101_" + quarter + ".pdf"
                driver.execute_script(r"viewProcessing('../pps/default.aspx?pp=PDF&ppstype=HeadlineWithArticleTOC',true);resetAction(); if(window.event)window.event.cancelBubble = true;return false;")
                time.sleep(5)
                for filename in os.listdir(prefs['download.default_directory']):
                    f = os.path.join(prefs['download.default_directory'], filename)
                    # checking if it is a file
                    if os.path.isfile(f):
                        os.replace(f, storage_path + "\\" + new_file_name)
                        status = True
                        reason = ""
                    else:
                        status = False
                        reason = "Too Many Files"
                        
        except Exception as e:
            status = False
            line_no = sys.exc_info()[-1].tb_lineno
            reason = " LineNo: " + str(line_no) + " Error: " + str(e)
        downloads_tracker['company'].append(companies_list.iloc[i,0])
        downloads_tracker['from_date'].append("2008/01/01")
        downloads_tracker['to_date'].append("2024/01/01")
        downloads_tracker['status'].append(status)
        downloads_tracker['reason'].append(reason)

In [25]:
output_result = pd.DataFrame(downloads_tracker)
output_result.to_csv("run2.csv")